# Fastapi

`fastapi` is a python library that allows you to build APIs on top of python.

Find out more here:

- [Introductory and tutorials on fastapi](https://fastapi.tiangolo.com/learn/).

In [4]:
import requests

## Run application

To ensure all notebooks on this site are runnable, I've implemented a solution using Docker containers. This approach allows for efficient execution and prevents notebook cell stacking, providing a seamless experience.  Below is a comprehensive list of requirements for running the application.

For detailed instructions on running a FastAPI application, please refer to the [specific page](fastapi/run_application.ipynb).

---

If you haven't already built a container with your FastAPI application, you'll need to do so. 

In [ ]:
!docker build -t fastapi_experiment fastapi/run_application_files/dockerfile


Create a file containing the application you want to play with. 

In [10]:
%%writefile fastapi/run_application_files/get_started.py
from fastapi import FastAPI

my_first_app = FastAPI()

@my_first_app.get("/")
def say_hello():
    return "hello"

Overwriting fastapi/run_application_files/get_started.py


Now you need to start the container.  There are a few important features to consider: 

* **`-v` option:**  Specifies the location of the file containing your program, allowing you to substitute a specific file.
* **Command to execute:** `uvicorn --host 0.0.0.0 --reload get_started:my_first_app`
    * **`--host 0.0.0.0`:**  Makes the API accessible from the host.
    * **`--reload`:**  Automatically applies changes to the API whenever you modify the application file. 


In [24]:
!docker run --rm -itd\
    --name test_container\
    -v ./fastapi/run_application_files/get_started.py:/get_started.py\
    -p 8000:8000 \
    fastapi_experiment \
    uvicorn --host 0.0.0.0 --reload get_started:my_first_app >/dev/null

Now you can test that everything is working correctly by making a request to the newly created API. 

In [12]:
requests.get("http://localhost:8000/").content

b'"hello"'

We received a response that matches the code we just wrote.

Finally, don't forget to stop the container when you're finished. 

In [14]:
!docker stop test_container

test_container


## Responses

FastAPI handles API output, returning Python objects or using special wrappers. Output annotations also play a role. For more details check [special page](fastapi/return_values.ipynb).

---

The following example shows an API that returns a simple Python dictionary as its output. 

In [25]:
%%writefile fastapi/run_application_files/get_started.py
from fastapi import FastAPI

my_first_app = FastAPI()

@my_first_app.get("/")
def get_dict():
    return {"a": "value", "b": 32}

Overwriting fastapi/run_application_files/get_started.py


If we request this API, we would receive the corresponding JSON data in response.

In [37]:
requests.get("http://localhost:8000").content

b'{"a":"value","b":32}'

## Cache

A common way to optimize applications is by using caching, which involves storing the outputs of certain processes for a set period. For FastAPI, the [`fastapi_cache`](https://github.com/long2ice/fastapi-cache) library provides a convenient way to implement caching, simplifying the process significantly.

---

This example demonstrates how to use `fastapi_cache` with a Redis database. The following code snippet defines a Compose file that sets up a FastAPI application and a Redis database.

In [1]:
%%writefile fastapi_files/docker-compose.yml
services:
  fast_api:
    image: fastapi_experiment
    container_name: cache_example_api
    volumes:
      - ./app.py:/app.py
    command: /bin/sh -c "\
      pip3 install redis fastapi-cache2[redis] && \
      uvicorn --host 0.0.0.0 --reload app:app"
    network_mode: host
    stdin_open: true
    tty: true
  redis:
    image: redis:7.4.0
    container_name: cache_example_redis
    network_mode: host
    command: redis-server --port 6380

Overwriting fastapi_files/docker-compose.yml


The following code implements a simple FastAPI application that generates a random number for a given user. We'll use this application to test if the returned values are cached in Redis. 

In [2]:
%%writefile fastapi_files/app.py
from random import random
from collections.abc import AsyncIterator

from fastapi import FastAPI

from fastapi_cache import FastAPICache
from fastapi_cache.decorator import cache
from fastapi_cache.backends.redis import RedisBackend

from redis import asyncio as aioredis

def lifespan(_: FastAPI) -> AsyncIterator[None]:
    redis = aioredis.from_url("redis://localhost:6380")
    FastAPICache.init(RedisBackend(redis), prefix="fastapi-cache")
    yield

app = FastAPI(lifespan=lifespan)

@app.get("/{id}")
@cache(expire=600)
def index(id: int):
    return random()

Overwriting fastapi_files/app.py


The following cell starts the application. 

In [3]:
%%bash
cd fastapi_files/
docker compose up -d &> /dev/null

Now you can try making requests to the application.  Different URLs will return different values. 

In [7]:
!curl localhost:8000/10
!curl localhost:8000/20

0.38783875657759437

0.9900099017385098

However, requests to the same URL will return the same values as before, because the results have been cached from previous calls.

In [9]:
!curl localhost:8000/10
!curl localhost:8000/20

0.38783875657759437

0.9900099017385098

Let's check if the cached values are present in the Redis database.

In [10]:
!redis-cli -h localhost -p 6380 keys '*'

1) "fastapi-cache::5511deb9b96a93779726aed3f9e4d8bc"
2) "fastapi-cache::c2ab82769e6470b9ef90fa0de4fbb612"


We see two keys that begin with the string "fastapi-cache", just as we specified in the `prefix` argument. The following Python code prints the values of all keys available in the Redis database. 

In [11]:
import redis

r = redis.Redis(host='localhost', port=6380, db=0)

matching_keys = []
cursor = '0'
while cursor != 0:
    cursor, keys = r.scan(cursor=cursor, match='fastapi-cache*')
    matching_keys.extend(keys)

if matching_keys:
    values = r.mget(matching_keys)
    for key, value in zip(matching_keys, values):
        print(f"{key.decode('utf-8')}: {value.decode('utf-8')}")

fastapi-cache::5511deb9b96a93779726aed3f9e4d8bc: 0.38783875657759437
fastapi-cache::c2ab82769e6470b9ef90fa0de4fbb612: 0.9900099017385098


The values retrieved from Redis match the values we received from the API responses.

Don't forget to clean the environment after all.

In [12]:
%%bash
cd fastapi_files/
docker compose down &> /dev/null